In [81]:
import snowflake.connector
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

# function to remove the PII data
def remove_field(obj, field_name):
    if isinstance(obj, dict):
        # Create a new dict without the target field
        return {k: remove_field(v, field_name) for k, v in obj.items() if k.lower() != field_name.lower()}
    elif isinstance(obj, list):
        # If it's a list, recurse into each item
        return [remove_field(item, field_name) for item in obj]
    else:
        # Base case: neither dict nor list
        return obj

# connect to snowflake
conn = snowflake.connector.connect(
    user='YAQIWU11235',
    password='nut3yub2tzh7MPA.dnc',
    account='MCB49099-KHB29787',
    warehouse='OSCILAR_TO_SNOWFLAKE',
    database = 'FLEXCAR_OSCILAR',
    schema = 'OSCILAR_SCHEMA'
    )

cur = conn.cursor()

try:
    # Fetch id and JSON column
    cur.execute("SELECT subscriber_id, created_at, response FROM oscilar_response where created_at > '2023-12-06 09:26:06' and response is not null limit 200 ")
    rows = cur.fetchall()

    update_sql = "UPDATE oscilar_response SET cleaned_response = parse_json(%s) WHERE subscriber_id = %s and TO_VARCHAR(created_at, 'YYYY-MM-DD HH24:MI:SS') = left(%s, 19)"

    for subscriber_id, created_at, response in rows:
        json_string = json.loads(response)
        cleaned_json = remove_field(json_string, 'ssn')
        cleaned_json_str = json.dumps(cleaned_json)
        # print(cleaned_json)
        print(f"Updating id={subscriber_id} and created_at={created_at} with cleaned JSON")
        cur.execute(update_sql, (cleaned_json_str, subscriber_id, created_at))

    conn.commit()

    # cur.execute(f"SELECT subscriber_id, created_at, response FROM oscilar_response where subscriber_id = {subscriber_id}")
    # rows = cur.fetchall()
    # print(rows)

finally:
    cur.close()
    conn.close()

Updating id=8d90fbc8-d1ba-4e1e-9c8a-c566e3914948 and created_at=2023-12-28 17:10:11.981298-08:00 with cleaned JSON
Updating id=ea9891eb-ba94-474a-8e08-519702242a8b and created_at=2023-12-28 17:14:39.815409-08:00 with cleaned JSON
Updating id=c3f73850-7fb8-47ff-b916-0d4a33175ecb and created_at=2023-12-28 17:16:47.596671-08:00 with cleaned JSON
Updating id=c3f73850-7fb8-47ff-b916-0d4a33175ecb and created_at=2023-12-28 17:17:13.773587-08:00 with cleaned JSON
Updating id=e38b9276-0da7-4b91-983d-8695596ed303 and created_at=2023-12-28 17:17:38.791283-08:00 with cleaned JSON
Updating id=023296fd-3914-4cd2-8b0b-f30d745fd95b and created_at=2023-12-28 17:26:56.527766-08:00 with cleaned JSON
Updating id=023296fd-3914-4cd2-8b0b-f30d745fd95b and created_at=2023-12-28 17:27:19.142736-08:00 with cleaned JSON
Updating id=62d951eb-6143-4d2a-b4bc-c05dddb163ef and created_at=2023-12-28 17:36:24.552234-08:00 with cleaned JSON
Updating id=1551dcf3-0688-47c9-ab13-671f36436e66 and created_at=2023-12-28 17:41

In [42]:
json_string

{'errors': [],
 'integrations': [{'name': 'Raw_Segments_DAI_2',
   'response': {'confidence': 1, 'predicted_Raw_Segment': 'D'}},
  {'name': 'SentiLink_ID',
   'response': {'application': {'city': 'Riverdale',
     'dob': '1974-03-27',
     'first_name': 'KESIHA',
     'last_name': 'HOUSTON',
     'ssn': '380-86-1903',
     'state_code': 'GA'},
    'confidence_level': 'very_low',
    'environment': 'PROD',
    'latency_ms': 48,
    'timestamp': '2023-12-06T17:25:32.496821798Z',
    'transaction_id': '01HH02KN-50HQ-B8Z78BGS',
    'updated_fields': ['ssn']}},
  {'name': 'ThomsonReuters',
   'response': {'EIDVPersonSearchResult': {'EIDVPersonSearchRequest': {'Criteria': {'BirthDate': {'Day': '27',
        'Month': '03',
        'Year': '1974'},
       'City': 'Riverdale',
       'DriversLicense': '055300036',
       'FirstName': 'KESIHA',
       'LastName': 'HOUSTON',
       'PhoneNumber': '6786734682',
       'SSN': '380-86-1903',
       'State': 'GA',
       'Street': '5787 Eagles Feathe

In [2]:
df['combined_set'] = df.iloc[:, -17:].apply(lambda row: set(row.values), axis=1)

for i in df['combined_set']:
    print(len(i))


1
10
10
10
10
9
9
10
10
9
10
10
10
9
9
10
10
10
9
9
10
10
9
10
10
9
10
10
10
10
10
10
10
8
10
9
10
9
10
10
10
10
10
4
10
10
10
1
10
10
10
10
10
10
10
10
10
10
10
10
10
9
10
10
9
10
10
10
10
10
10
10
10
9
10
10
10
10
10
9
9
10
10
10
9
10
9
10
9
10
10
9
7
10
10
9
9
9
9
10
10
10
9
9
10
10
10
10
8
10
9
9
8
10
10
10
10
9
10
9
10
10
10
10
8
9
10
10
10
10
10
10
10
3
3
3
3
9
9
10
9
10
10
9
9
10
10
10
9
10
9
10
9
10
9
10
9
10
10
9
8
9
9
9
10
9
9
9
9
8
9
10
10
10
9
9
10
9
10
10
10
10
8
10
9
9
10
9
10
10
9
9
9
8
9
9
9
10
10
10


In [5]:
for i in df['combined_set']:
    print(len(i))

9
10
9
8
10
9
10
10
8
10


In [3]:
import snowflake.connector
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

conn = snowflake.connector.connect(
    user='YAQIWU11235',
    password='nut3yub2tzh7MPA.dnc',
    account='MCB49099-KHB29787',
    warehouse='OSCILAR_TO_SNOWFLAKE',
    database = 'FLEXCAR_OSCILAR',
    schema = 'OSCILAR_SCHEMA'
    )

columns = ["Subscriber Id",
           "Created At",
            "ClearID_TotalScore",
            "RiskInform_TotalScore",
            "RiskInform_AddressFlags_TotalScore",
            "RiskInform_CriminalFlags_TotalScore",
            "RiskInform_Custom_TotalScore",
            "RiskInform_SingleRiskIndicators_TotalScore",
            "RiskInform_SyntheticIdentity_TotalScore"
        ]

q = f"""
            select 
                subscriber_id,
                response
            from 
                oscilar_response
            where created_at > '2023-12-06 09:26:06'
            and response is not null    
            limit 1        
"""

# get the column names
# q_col = "show columns in tr_attributes;"

with conn.cursor() as cur:
    data = cur.execute(q).fetchall()

df = pd.DataFrame(data)


In [38]:
response

'{\n  "errors": [],\n  "integrations": [\n    {\n      "name": "ThomsonReuters",\n      "response": {\n        "EIDVPersonSearchResult": {\n          "EIDVPersonSearchRequest": {\n            "Criteria": {\n              "BirthDate": {\n                "Day": "18",\n                "Month": "10",\n                "Year": "2001"\n              },\n              "City": "Forest Park",\n              "DriversLicense": "061492145",\n              "FirstName": "Joshua",\n              "LastName": "Bangham",\n              "PhoneNumber": "4043996431",\n              "SSN": "667-14-3102",\n              "State": "GA",\n              "Street": "715 Catherine Street",\n              "ZipCode": "30297"\n            },\n            "EIDVName": "Flexcar Person ID Confirm",\n            "EIDVVersion": "1"\n          },\n          "NormalizedCriteria": {\n            "BirthDate": "10/18/2001",\n            "City": "FOREST PARK",\n            "DriversLicense": "061492145",\n            "FirstName": "

In [26]:
import json
json_string = json.loads(df.iloc[0,1])
type(json_string)

dict

In [31]:
json_string

{'errors': [],
 'integrations': [{'name': 'ThomsonReuters',
   'response': {'EIDVPersonSearchResult': {'EIDVPersonSearchRequest': {'Criteria': {'BirthDate': {'Day': '31',
        'Month': '03',
        'Year': '1990'},
       'City': 'Stone Mountain',
       'DriversLicense': '059547714',
       'FirstName': 'Alexandria',
       'LastName': 'Simmons',
       'PhoneNumber': '4704948337',
       'SSN': '420-35-0203',
       'State': 'GA',
       'Street': '4900 Central Dr',
       'ZipCode': '30083'},
      'EIDVName': 'Flexcar Person ID Confirm',
      'EIDVVersion': '1'},
     'NormalizedCriteria': {'BirthDate': '03/31/1990',
      'City': 'STONE MOUNTAIN',
      'DriversLicense': '059547714',
      'FirstName': 'Alexandria',
      'LastName': 'Simmons',
      'SSN': '420-35-0203',
      'State': 'GA',
      'Street': '4900 CENTRAL DR',
      'ZipCode': '30083'},
     'PersonEntities': {'PersonEntity': {'AdditionalPhoneNumbers': {'AdditionalPhoneNumber': [{'DocumentIdentifierInfo': [{'

In [29]:
cleaned_response = remove_field(json_string, 'SSN')
cleaned_response = remove_field(cleaned_response, 'ssn')

In [30]:
cleaned_response

{'errors': [],
 'integrations': [{'name': 'ThomsonReuters',
   'response': {'EIDVPersonSearchResult': {'EIDVPersonSearchRequest': {'Criteria': {'BirthDate': {'Day': '31',
        'Month': '03',
        'Year': '1990'},
       'City': 'Stone Mountain',
       'DriversLicense': '059547714',
       'FirstName': 'Alexandria',
       'LastName': 'Simmons',
       'PhoneNumber': '4704948337',
       'State': 'GA',
       'Street': '4900 Central Dr',
       'ZipCode': '30083'},
      'EIDVName': 'Flexcar Person ID Confirm',
      'EIDVVersion': '1'},
     'NormalizedCriteria': {'BirthDate': '03/31/1990',
      'City': 'STONE MOUNTAIN',
      'DriversLicense': '059547714',
      'FirstName': 'Alexandria',
      'LastName': 'Simmons',
      'State': 'GA',
      'Street': '4900 CENTRAL DR',
      'ZipCode': '30083'},
     'PersonEntities': {'PersonEntity': {'AdditionalPhoneNumbers': {'AdditionalPhoneNumber': [{'DocumentIdentifierInfo': [{'DocumentIdentifier': 'Ie92811d1906711ea80afece799150095',